In [4]:
import numpy as np

**✅ 1) Lọc theo thời gian trong pandas (dùng cột timestamp)**

Lọc theo danh sách timestamp cần giữ

In [1]:
def filter_by_timestamps_df(df, keep_timestamps):
    if keep_timestamps is None:
        return df

    mask = df['timestamp'].isin(keep_timestamps)
    return df[mask]


Lọc theo khoảng thời gian

In [2]:
def filter_by_range_df(df, t_min, t_max):
    mask = (df['timestamp'] >= t_min) & (df['timestamp'] <= t_max)
    return df[mask]


**✅ 2) Lọc theo thời gian trong NumPy**

Lọc theo danh sách timestamp

In [6]:
def filter_by_timestamps_np(data, timestamps, keep_timestamps):
    # Chuyển danh sách timestamp cần giữ sang dạng NumPy array
    # để np.isin() có thể so sánh vectorized nhanh hơn
    keep_timestamps = np.array(keep_timestamps)

    # Tạo một mask (mảng True/False) có cùng chiều với 'timestamps'
    # True  nếu timestamps[i] nằm trong keep_timestamps
    # False nếu không
    mask = np.isin(timestamps, keep_timestamps)

    # Áp mask lên chiều 0 của 'data'
    # => Chỉ giữ lại các frame/mốc thời gian mà mask == True
    return data[mask]



Lọc theo khoảng thời gian

In [5]:
def filter_by_range_np(data, timestamps, t_min, t_max):
    mask = (timestamps >= t_min) & (timestamps <= t_max)
    return data[mask]


ví dụ

In [7]:
# Dữ liệu chính: 5 thời điểm, mỗi thời điểm là 2x2
data = np.array([
    [[1, 2],
     [3, 4]],   # time 0

    [[10, 20],
     [30, 40]], # time 1

    [[100, 200],
     [300, 400]], # time 2

    [[1000, 2000],
     [3000, 4000]], # time 3

    [[7, 8],
     [9, 10]]  # time 4
])

# timestamps tương ứng cho từng time step
timestamps = np.array([3, 5, 10, 12, 20])

In [8]:
print(data.shape)
print(timestamps.shape)


(5, 2, 2)
(5,)


In [9]:
def filter_by_range_np(data, timestamps, t_min, t_max):
    # Tạo mask thời gian
    mask = (timestamps >= t_min) & (timestamps <= t_max)

    # Trả về các phần tử theo mask
    return data[mask], timestamps[mask]


filtered_data, filtered_timestamps = filter_by_range_np(data, timestamps, 5, 12)


In [11]:
display(data)
display(filtered_data)
display(filtered_timestamps)

array([[[   1,    2],
        [   3,    4]],

       [[  10,   20],
        [  30,   40]],

       [[ 100,  200],
        [ 300,  400]],

       [[1000, 2000],
        [3000, 4000]],

       [[   7,    8],
        [   9,   10]]])

array([[[  10,   20],
        [  30,   40]],

       [[ 100,  200],
        [ 300,  400]],

       [[1000, 2000],
        [3000, 4000]]])

array([ 5, 10, 12])

In [ ]:
# timestamps = [3, 5, 10, 12, 20]
# t_min = 5
# t_max = 12
#
# mask = [False, True, True, True, False]

# data[mask]  “Hãy giữ lại tất cả data[i] mà mask[i] == True.”

In [ ]:
# import numpy as np
#
# data = np.array([100, 200, 300, 400, 500])
# mask = np.array([False, True, True, False, True])
#
# print(data[mask])
# [200 300 500]


In [12]:
'''
🔥 2) Cách ghép TẤT CẢ DATA thành một, theo chiều thời gian
❗ Bạn muốn:

Có nhiều file CSV hoặc nhiều mảng data tương ứng các thời điểm.

Các file đều có trường timestamp.

Bạn muốn ghép lại thành 1 bảng duy nhất, và chỉ giữ các timestamps xuất hiện ở tất cả file.

→ Đây gọi là inner join theo thời gian.
'''

'\n🔥 2) Cách ghép TẤT CẢ DATA thành một, theo chiều thời gian\n❗ Bạn muốn:\n\nCó nhiều file CSV hoặc nhiều mảng data tương ứng các thời điểm.\n\nCác file đều có trường timestamp.\n\nBạn muốn ghép lại thành 1 bảng duy nhất, và chỉ giữ các timestamps xuất hiện ở tất cả file.\n\n→ Đây gọi là inner join theo thời gian.\n'

**🔧 4) Code Python để ghép theo timestamp**

In [ ]:
import pandas as pd
import glob

def load_and_merge(path_pattern):
    files = glob.glob(path_pattern)

    dfs = []
    for f in files:
        df = pd.read_csv(f)
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        dfs.append(df)

    # Bắt đầu merge từ file đầu tiên
    df_merged = dfs[0]

    for i in range(1, len(dfs)):
        df_merged = df_merged.merge(
            dfs[i],
            on='timestamp',
            how='inner'    # chỉ giữ timestamp chung
        )

    return df_merged

df_final = load_and_merge("file*.csv")
print(df_final)


À, giờ tôi hiểu ý bạn: bạn muốn chuẩn hóa lại trục thời gian giữa nhiều dataset, theo cách:

Gán mỗi timestamp một số nguyên liên tiếp (0, 1, 2…) → gọi là mapping timestamp → time index.

Khi ghép dữ liệu lại, mọi dataset đều dùng cùng time index.

Với mỗi time index, tất cả dữ liệu ở thời điểm đó đều phải có đầy đủ.

Nếu một dataset không có dữ liệu ở timestamp đó → loại bỏ (inner join).

Mình sẽ viết hướng dẫn + code minh họa đầy đủ từ đầu đến cuối.

1️⃣ Bước 1: Chuẩn hóa timestamp

In [14]:
import pandas as pd

# Ví dụ dữ liệu
df1 = pd.DataFrame({
    'timestamp': ['2020-01-01','2020-01-02','2020-01-03'],
    'value1':[10, 20, 30]
})

df2 = pd.DataFrame({
    'timestamp': ['2020-01-01','2020-01-03','2020-01-04'],
    'value2':[100, 300, 400]
})

df3 = pd.DataFrame({
    'timestamp': ['2020-01-02','2020-01-03','2020-01-04'],
    'value3':[2000, 3000, 4000]
})


2️⃣ Bước 2: Gán time index cho các timestamp

In [17]:
# 1. Lấy tất cả timestamp xuất hiện trong các df
all_timestamps = pd.concat([df1['timestamp'], df2['timestamp'], df3['timestamp']])
all_timestamps = pd.to_datetime(all_timestamps)
unique_timestamps = all_timestamps.sort_values().unique()

# 2. Tạo mapping: timestamp → time index (0,1,2,…)
timestamp_to_index = {ts:i for i, ts in enumerate(unique_timestamps)}
for r in timestamp_to_index:
    print(r)
# print(timestamp_to_index)


2020-01-01 00:00:00
2020-01-02 00:00:00
2020-01-03 00:00:00
2020-01-04 00:00:00


3️⃣ Bước 3: Thêm cột time_index vào từng dataset

In [19]:
for df in [df1, df2, df3]:
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['time_index'] = df['timestamp'].map(timestamp_to_index)

display(df)

,timestamp,value3,time_index
0,2020-01-02,2000,1
1,2020-01-03,3000,2
2,2020-01-04,4000,3


4️⃣ Bước 4: Ghép dữ liệu theo time_index (inner join)

In [20]:
df_merged = df1.merge(df2, on='time_index', how='inner') \
               .merge(df3, on='time_index', how='inner')

# Bỏ cột timestamp dư (lấy của df1)
df_merged = df_merged[['time_index', 'value1', 'value2', 'value3']]
print(df_merged)


   time_index  value1  value2  value3
0           2      30     300    3000


In [1]:
import numpy as np
x = np.load("../csv_data/old/x_hatinh.npy")
y = np.load("../csv_data/old/y_hatinh.npy")

In [2]:
print(x.shape)
print(y.shape)

(1223, 12, 21, 34)
(1223, 21, 34)
